In [1]:
%discover
%connect test-stm32 serial


test-stm32                     serial:///dev/cu.usbmodem208E307542521  
Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521


In [1]:
import socket
import network

ip = network.WLAN(network.STA_IF).ifconfig()[0]

s = socket.socket()
ai = socket.getaddrinfo("0.0.0.0", 80)
addr = ai[0][-1]

s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(addr)
s.listen(2)

print("waiting for connection at http://{}".format(ip))
for i in range(6):
    ss, addr = s.accept()
    print("connected from", addr)
    req = ss.recv(1024)
    if i < 4:
        ss.send('Hello from Airlift, serving request # {}!'.format(i+1).encode())
    else:
        ss.send(b'So long! Server shutting down.')
    ss.close()
s.close()

print("DONE")

waiting for connection at http://10.39.40.129
connected from ('10.39.40.114', 52662)
connected from ('10.39.40.114', 52663)
connected from ('10.39.40.114', 52665)
connected from ('10.39.40.114', 52666)
connected from ('10.39.40.114', 52667)
connected from ('10.39.40.114', 52668)
DONE


In [1]:
%discover
%connect test-stm32 serial

import socket, select
from urpc import eq_

def handle(cs, n):
    try:
        poller = select.poll()
        poller.register(cs, select.POLLIN)
        for s, ev in poller.poll(-1):
            req = cs.recv(1043)
            if req == b'':
                print("Client disconnected")
                return
            cs.send('Hello! {}-{}'.format(n, i).encode())
    except Exception as e:
        print("***", e)
    finally:
        print("close cs")
        cs.close()
            
def serve(cb, port=80, backlog=5):
    poller = select.poll()
    
    ai = socket.getaddrinfo("0.0.0.0", port)
    addr = ai[0][-1]

    ss = socket.socket()
    try:
        ss.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        ss.setblocking(False)
        ss.bind(addr)
        ss.listen(backlog)

        poller.register(ss, select.POLLIN)

        for i in range(3):
            for s, ev in poller.poll(-1):
                assert eq_(s, ss), "server socket mismatch"
                cs, addr = s.accept()
                print("connected from", addr)
                # req = cs.recv(1043)
                # cs.send(b"blabla")
                # cs.close()
                cb(cs, i)
    finally:
        ss.close()
            
import network
print("start ...")
ip = network.WLAN(network.STA_IF).ifconfig()[0]
print("http://{}".format(ip))

serve(handle)

test-stm32                     serial:///dev/cu.usbmodem208E307542521  
test-stm32-cop                 serial:///dev/cu.usbserial-0143520E     
Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521
start ...
http://10.39.40.129
connected from ('10.39.40.114', 52828)
close cs
connected from ('10.39.40.114', 52829)
close cs
connected from ('10.39.40.114', 52830)
close cs


In [1]:
import uasyncio as asyncio
import socket

async def handler(reader, writer):
    try:
        req = await reader.read()
        print("got req", req)
        writer.write(b'response!')
        await writer.drain()
        print("handler done")
    finally:
        # close streams
        await reader.closed()
        await writer.closed()


async def main(addr, port):
    await asyncio.start_server(handler, addr, port)
    asyncio.get_event_loop().run_forever()
    

ai = socket.getaddrinfo("0.0.0.0", port)
addr = ai[0][-1][0]
port = 80

asyncio.run(main(addr, port))




Traceback (most recent call last):
  File "<stdin>", line 26, in <module>
  File "uasyncio/core.py", line 1, in run
  File "uasyncio/core.py", line 1, in run_until_complete
  File "uasyncio/core.py", line 1, in wait_io_event
KeyError: 536921152


In [ ]:
class IOQueue:
    def __init__(self):
        self.poller = select.poll()
        self.map = {}  # maps id(stream) to [task_waiting_read, task_waiting_write, stream]

    def _enqueue(self, s, idx):
        if id(s) not in self.map:
            entry = [None, None, s]
            entry[idx] = cur_task
            self.map[id(s)] = entry
            self.poller.register(s, select.POLLIN if idx == 0 else select.POLLOUT)
        else:
            sm = self.map[id(s)]
            assert sm[idx] is None
            assert sm[1 - idx] is not None
            sm[idx] = cur_task
            self.poller.modify(s, select.POLLIN | select.POLLOUT)
        # Link task to this IOQueue so it can be removed if needed
        cur_task.data = self

    def _dequeue(self, s):
        del self.map[id(s)]
        self.poller.unregister(s)

    def queue_read(self, s):
        self._enqueue(s, 0)

    def queue_write(self, s):
        self._enqueue(s, 1)

    def remove(self, task):
        while True:
            del_s = None
            for k in self.map:  # Iterate without allocating on the heap
                q0, q1, s = self.map[k]
                if q0 is task or q1 is task:
                    del_s = s
                    break
            if del_s is not None:
                self._dequeue(s)
            else:
                break

    def wait_io_event(self, dt):
        for s, ev in self.poller.ipoll(dt):
            sm = self.map[id(s)]
            # print('poll', s, sm, ev)
            if ev & ~select.POLLOUT and sm[0] is not None:
                # POLLIN or error
                _task_queue.push_head(sm[0])
                sm[0] = None
            if ev & ~select.POLLIN and sm[1] is not None:
                # POLLOUT or error
                _task_queue.push_head(sm[1])
                sm[1] = None
            if sm[0] is None and sm[1] is None:
                self._dequeue(s)
            elif sm[0] is None:
                self.poller.modify(s, select.POLLOUT)
            else:
                self.poller.modify(s, select.POLLIN)

                
_io_queue = IOQueue()